<a href="https://colab.research.google.com/github/cadv1984/TELECOMX-2/blob/main/TELECOM_X_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción del Archivo

Este código comienza con importar las librerías necesarias como pandas, numpy entre otras. Se carga archivo del proyecto anterior y luego se realiza una limpieza inicial.

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns



In [17]:
#carga de datos

try:
  df = pd.read_csv('/content/datos_tratados.csv')
except FileNotFoundError:
  print("El archivo no se encuentra en la ruta especificada.")
except Exception as e:
  print(f"Ocurrió un error al cargar el archivo: {e}")



In [18]:
#limpiar datos

df = df.drop('customerID', axis=1)
df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'], errors='coerce')
df = df.dropna()

print ('Formato de los datos luego de la limpieza:')
print(df.info())

# validar proporción de CHURN

churn_porportion = df['Churn'].value_counts(normalize=True)
print('\nProporción de clientes que cancelaron servicio')
print(churn_porportion)

Formato de los datos luego de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Churn                      7043 non-null   object 
 1   customer.gender            7043 non-null   object 
 2   customer.SeniorCitizen     7043 non-null   int64  
 3   customer.Partner           7043 non-null   object 
 4   customer.Dependents        7043 non-null   object 
 5   customer.tenure            7043 non-null   int64  
 6   phone.PhoneService         7043 non-null   object 
 7   phone.MultipleLines        7043 non-null   object 
 8   internet.InternetService   7043 non-null   object 
 9   internet.OnlineSecurity    7043 non-null   object 
 10  internet.OnlineBackup      7043 non-null   object 
 11  internet.DeviceProtection  7043 non-null   object 
 12  internet.TechSupport       7043 non-null   object 
 13  internet.S

# Preprocesamiento de datos y preparación para el modelado


Se definen las variables categóricas y numéricas para aplicar diferentes transformaciones. La variable objetivo churn se mapea a valores binarios (0 y 1). Luego, se utiliza ColumnTransforme para aplicar OneHotEncoder a las variables categóricas y StandardScaler (normalización) a las variables numéricas. Esta metodología asegura que cada  tipo de dato reciba el procesamiento adecuado antes de ser alimentado a los modelos. Finalmente, los datos se dividen en conjuntos de entrenamiento y prueba.

In [19]:
# definir varibles categóricas y numéricas
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('Churn')  # Ahora 'Churn' sí está en la lista para ser eliminado.
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# mapear variable objetivo a valores binarios
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

#separa variables predictoras (X) y variable objetivo (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

#separa los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#one-hot para variables categóricas
# normalización (StandarScaler) para variables numéricas
preprocessor_lr = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
    )

print('Dimensiones de los conjuntos de entrenamiento y prueba:')
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)
#


Dimensiones de los conjuntos de entrenamiento y prueba:
X_train: (5634, 20)
X_test: (1409, 20)
y_train: (5634,)
y_test: (1409,)


# Creación y Evaluación de Modelos

In [21]:
# modelo 1: regresión logística con normalización

pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor_lr),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)

print('Métricas del modelo:')
print(f'Exactitud:{accuracy_score(y_test, y_pred_lr):.4f}')
print(f'Precisión:{precision_score(y_test,y_pred_lr):.4f}')
print(f'Recall:{recall_score(y_test, y_pred_lr):.4f}')
print(f'F1-Score:{f1_score(y_test, y_pred_lr):.4f}')
print(f'\n Matriz de Confusión:\n',confusion_matrix(y_test, y_pred_lr))

#


Métricas del modelo:
Exactitud:0.7935
Precisión:0.6352
Recall:0.5214
F1-Score:0.5727

 Matriz de Confusión:
 [[923 112]
 [179 195]]
